# Scrittopoli - Organizzazione partite

In [1]:
%matplotlib inline

# Import the necessary packages and modules
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.dates  as mpdt
import matplotlib.ticker as mptk

import numpy as np
import pytz
mytz=pytz.timezone("Europe/Rome")
import pandas as pd
import datetime as dt
import math
import importlib
import random
import itertools
import csv
import mylib.spreadsheet as ms
import mylib.elo as melo


In [2]:
day="20171209_192536"
calendario_csv="data/calendario_%s.csv" % day
gironi_csv="data/gironi_%s.csv" % day
tabellone_csv="data/tabellone_%s.csv" % day
ranking_csv="data/ranking_%s.csv" % day
scrittopoli_csv="data/scrittopoli_%s.csv" % day
elo_giocatori_csv="data/elo_giocatori_%s.csv" % day
elo_squadre_csv="data/elo_squadre_%s.csv" % day

calendario=pd.read_csv(calendario_csv,sep=":",quotechar='"',header=0,index_col=1).sort_index()
calendario=calendario.set_index(["girone","partita"], append=True)

gironi=pd.read_csv(gironi_csv,sep=":",quotechar='"',header=0)

tabellone=pd.read_csv(tabellone_csv,sep=",",quotechar='"',header=0,index_col=0)
elo_giocatori=pd.read_csv(elo_giocatori_csv,sep=",",quotechar='"',header=0,index_col=0)

ranking=pd.read_csv(ranking_csv,sep=",",quotechar='"',header=0)
ranking=ranking[ranking["squadra"]!="(riposo)"]
ranking=ranking.set_index("squadra")

scrittopoli=pd.read_csv(scrittopoli_csv,sep=",",quotechar='"',header=0)
scrittopoli=scrittopoli.set_index(["squadra","giocatore"])

elo_squadre=pd.read_csv(elo_squadre_csv,sep=",",quotechar='"',header=0)
elo_squadre=elo_squadre.set_index(["squadra","giocatore"])



## Come si svolgono gli incontri

Ogni capitano presenta la formazione (due titolari e una riserva). I due titolari vengono accoppiati agli avversari sulla base del punteggio Elo (i due più alti tra loro e i due più bassi tra loro).

Inseriscono i due racconti. Cose da controllare:
* link al racconto;
* tag;
* lunghezza del racconto;
* link al commento;
* commento valido.

Per la votazione ogni squadra presenta l'elenco dei voti da dare a quelle degli altri gironi. Per ogni partita danno quattro punteggi. Oltre a queste ci sono le penalità dell'arbitro.


In [3]:
calendario[calendario["squadra 2"]!="(riposo)"].sort_index()

squadra 1                      squadra 2
giornata girone partita                                                   
1        A      1               Green House                     I Bananari
         B      1             Living Stones                   La threesome
         C      1               D Euforiche             Retroguardia Trash
                2            the 3 BrOthers             Armata Brancaleone
2        A      1               Green House  Gli scrittori del quartierino
         B      1             Living Stones                    Le TreMende
         C      1               D Euforiche                 the 3 BrOthers
                2        Retroguardia Trash             Armata Brancaleone
3        A      2                I Bananari  Gli scrittori del quartierino
         B      2              La threesome                    Le TreMende
         C      1               D Euforiche             Armata Brancaleone
                2        Retroguardia Trash                 the 3 BrOthers

## Fase 1. Dalle formazioni ai match

In [4]:
squadre=scrittopoli.copy()
squadre=squadre.reset_index()[["squadra","giocatore"]].set_index("squadra")

# Versione random di prova, da sostituire con lettura della formazione via csv o spreadsheet
data=[]
for sq in squadre.index.unique():
    riserva=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
    titolari=list(squadre[squadre["giocatore"]!=riserva].loc[sq]["giocatore"])
    data.append([sq,riserva]+titolari)
formazioni=pd.DataFrame(data,columns=["squadra","riserva","titolare 1","titolare 2"])


In [5]:
formazioni=formazioni.set_index("squadra")
formazioni

,riserva,titolare 1,titolare 2
squadra,,,
Green House,Joyopi,Bango Skank,Plata
La threesome,Komorebi,Luna,mina99
I Bananari,Niko,simone volponi,massimopud
Armata Brancaleone,Skorpiòs,Ella F.,AdStr
Living Stones,Ljuset,Macleo,Vincenzo Iennaco
Retroguardia Trash,Eudes,AlexComan,Andrea28
Le TreMende,Emy,sefora,Thea
Gli scrittori del quartierino,Marty12,camparino,Federico72
the 3 BrOthers,AryaSophia,ettore70,ITG


In [6]:
def parametri_giornata(giornata,calendario,elo):
    rank=elo.copy().reset_index().set_index("giocatore")["rank %d" % (giornata-1)]
    cal=calendario[calendario["squadra 2"]!="(riposo)"].loc[1]
    return cal,rank

def sort_by_rank(rank,a,b):
    if rank[a]>rank[b]: return [a,b]
    if rank[a]<rank[b]: return [b,a]
    L=[a,b]
    random.shuffle(L)
    return L

def calcola_accoppiamenti(giornata,formazioni,calendario,elo_squadre):
    cal,rank=parametri_giornata(1,calendario,elo_squadre)
    data=[]
    for girone,partita in cal.index:
        for n in [1,2]:
            squadra=cal.loc[(girone,partita)]["squadra %d" % n]
            riserva=formazioni.loc[squadra]["riserva"]
            titolare_1=formazioni.loc[squadra]["titolare 1"]
            titolare_2=formazioni.loc[squadra]["titolare 2"]
            data.append([girone,partita,squadra,riserva]+sort_by_rank(rank,titolare_1,titolare_2))    
    X=pd.DataFrame(data,
                   columns=["girone","partita","squadra","riserva",
                            "match 1","match 2"])
    X=X.set_index(["girone","partita","squadra"]).sort_index()
    return X

accoppiamenti=calcola_accoppiamenti(1,formazioni,calendario,elo_squadre)
accoppiamenti


riserva      match 1           match 2
girone partita squadra                                                      
A      1       Green House             Joyopi  Bango Skank             Plata
               I Bananari                Niko   massimopud    simone volponi
B      1       La threesome          Komorebi       mina99              Luna
               Living Stones           Ljuset       Macleo  Vincenzo Iennaco
C      1       D Euforiche           Marcello       queffe              Kuno
               Retroguardia Trash       Eudes     Andrea28         AlexComan
       2       Armata Brancaleone    Skorpiòs      Ella F.             AdStr
               the 3 BrOthers      AryaSophia          ITG          ettore70

## Fase 2. Creazione del tabellone per voti e controlli

In [ ]:
## Fase 3. Lettura dei controlli e penalità
## Fase 4. Lettura del tabellone, calcolo dei risultati, della classifica e dei nuovi ranking
